In [1]:
## Written By Shamima Rashid, Jan 2022
## Jupytope: Computational Extraction of Viral Epitopes
## Nanyang Technological University, Singapore 639798
# ##############################################################################################################
# This tool is meant to run on PDB structures. It will extract structural properties for viral epitopes
# # Needed PDB files should first be downloaded, and split into single chains. 
# The files should be named PDBID_Chain.pdb
# See HA_H1N1/Chains, HA_H3N2/Chains and Spike/Chains for examples.
# Then, the Alignments will be automatically filled by Jupytope.
# It has been tested on HA of Influenza and Spike of SARS-CoV2 structures, respectively (see EXAMPLES below).
# ###############################################################################################################

# To be prepared before running Jupytope
######################################################################################################
#   0. Download PDB structures of viral antigens (including reference structure)
#      Extract individual chains, and name it as PDBID_Chain.pdb (eg. 1RU7_A.pdb for HA of H1N1)##      
######################################################################################################


# Change lines in this cell as needed, and run the notebook:

######################################################################################################
#   1. Specify directories
#   Specify the base directory that will contain antigen chains and fasta alignments
##   It is recommended to place these directories within the Jupytope folder
######################################################################################################
# EXAMPLES
# base_dir = "Spike/"
# base_dir ="HA_H1N1/"


base_dir = "HA_H3N2/"
chain_dir = base_dir+"Chains/" # Create directory Chains. Place PDB chain splitted files within this directory
aln_dir = base_dir + "Alignments/" #Create directory Alignments

###################################################################################################
# 2. Specify Input Files
# It is recommended to place these files within the same directory as this notebook
###################################################################################################
# EXAMPLES # Filename containing list of PDB IDs and Chains
# id_file = "SARSCoV2_PDBID.tsv" 
# id_file ="H1N1_PDBID.tsv" 

id_file ="H3N2_PDBID.tsv"
s1 = "\t" #specify the delimiter for id_file. 

# EXAMPLES  # Filename containing list of numbered Epitope Sites
# sitesin_fn = 'Sites7DZW_Num.txt'  
# sitesin_fn = 'Sites1RU7_Num.txt' 

sitesin_fn = 'Sites1HGD_Num.txt'
s2 = "\t" #specify the delimiter for sitesin_fn. 

###############################################################################################
# 3. Specify PDB ID and Chain of reference structure. Place its PDB file in chain_dir
##############################################################################################
# EXAMPLES
# struct_id1 = "7DZW" #SARSCoV2
# chain_id1 = "A"
# struct_id1 = "1RU7" #H1N1      
# chain_id1 = "A"

struct_id1 = "1HGD"  #H3N2     
chain_id1 = "A"



##########################################################################################
# 4. Specify output filename to write extracted structure-derived data  
# This file will be written to the same directory as the notebook
##########################################################################################
#EXAMPLES
#op_fn = 'Sites_final_SARSCoV2.txt'
# op_fn = 'Sites_final_H1N1.txt'

op_fn = 'Sites_final_H3N2.txt'

# EXAMPLES #specify the delimiter for op_fn
# c = "\t" # tsv
# c = " " # space
c = "," # csv 


In [2]:
import os
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.DSSP import DSSP
from Bio.PDB.StructureAlignment import StructureAlignment
from Bio.PDB.Polypeptide import is_aa
from Bio.PDB import HSExposure 
from Bio.PDB.NACCESS import NACCESS
from Bio.PDB import ResidueDepth #runs the msms program

from Bio import pairwise2 
#from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO
import re
import pandas as pd
import numpy as np

In [3]:
unmatched = 0 #counter for unmatched residues. Debug Mode.

In [4]:
#load the structure objects and get the model. 1RUYH and 1RU7A - done
def get_model(pdb_id, chain_id):    

    filename = chain_dir + pdb_id + "_" + chain_id + ".pdb"
    structure_id = pdb_id
   
    parser = PDBParser(PERMISSIVE=1, QUIET=1)

    if(not os.path.isfile(filename)):
        print ("No file %s, please check" %(filename))
    else:
        structure = parser.get_structure(structure_id, filename)
        model = structure[0] #get the first model        
                   
    return model  

In [5]:
def get_seq(chain, pdb_id):
    #the following modres are converted to standard aa
    from Bio.SeqUtils import seq1
    modres = {

    'CSO': 'CYS',
    'CSX': 'CYS',
    'LLP': 'LYS',
    'MSE': 'MET',
    'MLY': 'LYS',
    'PXU': 'PRO',
    'SEP': 'SER',
    'TPO': 'THR' 
    }
    SEQATOM = ""
    
    for residue in chain:
        res_id = residue.get_id()
        hetfield = res_id[0] #read 1st tuple element into variable hetfield
        res_name = residue.get_resname()
        if hetfield  == " ": #if not HETATM
            SEQATOM = SEQATOM + res_name
                    
        elif re.match(r'^H', hetfield):
            if res_name in modres:
                SEQATOM = SEQATOM + modres[res_name]

#            Uncomment for Exception Handling    
#            else: #All other MODRES checked for individual error files
#                 #print("MODRES %s was found in %s chain %s" %(res_name, pdb_id, chain)) #print to stdout
#                 print("MODRES %s was found in %s chain %s" %(res_name, pdb_id, chain), file = op ) #print to file
#         else:
#             print("Strange HETATOM", hetfield, "found")
#     #print(SEQATOM)
    SEQ = seq1(SEQATOM)
    return SEQ

In [6]:
def get_algn(seq1, struct_id1, chain_id1, seq2, struct_id2, chain_id2):
  
    fafile = aln_dir+struct_id1+chain_id1+"_"+ struct_id2+chain_id2+".fa"
    
    if not os.path.isfile(fafile): #create the file if it doesn't exist      
         
        algn = pairwise2.align.globalms(seq1, seq2, 3, -1, -2.9, -0.5)
        f2 = open(fafile, 'w')
        f2.write(">%s | Chain %s \n" %(struct_id1, chain_id1))
        f2.write(algn[0][0]+"\n")

        f2.write(">%s | Chain %s \n" %(struct_id2, chain_id2))
        f2.write(algn[0][1]+"\n")

        f2.close()

    return fafile

In [7]:
def get_map_res(fafile, chain1, chain2):

    algn=AlignIO.read(fafile, "fasta")

    s_algn = StructureAlignment(algn, chain1, chain2, 0, 1)
    map_res = s_algn.get_maps()


    #map_res[0] -> dictionary containing alignment of seq1 to seq2
    #to  (e.g. 1RU7_A, PR/8/34, 1RUY_H, 1930/swine)
    global unmatched #for tracking number of un-aligned residues.
    
    unmatched = 0
  
    for residue in map_res[0]: #Optimization - loop over "Sites" instead
        res_id1 = residue.get_id()
        resname1 = residue.get_resname() 
        if map_res[0].get(residue): #None object
            res_id2 = map_res[0].get(residue).get_id() #get() value for key
            resname2 = map_res[0].get(residue).get_resname()
            #print(res_id1[1], resname1, res_id2[1], resname2)
        else:
            res_id2 = ('','None','')
            unmatched += 1
                   
        for i, row in enumerate(Sites):
            if row[0] == res_id1[1]: #if matched to epitope site
                Sites[i][2] = res_id2[1]
                Sites[i][3] = res_id2 #struct2 res_id for property retrieval
       
    return Sites  

In [8]:
def get_dssp(model2, pdb_id, chain_id):
    "'dssp' is the name of the binary or executable"
    #run dssp for a given model and retrieve residue properties   
    model = model2
    pdb_fn = chain_dir + pdb_id + "_" + chain_id + ".pdb"
   
    dssp = DSSP(model, pdb_fn, dssp='dssp')  # acc_array = "Sander"(default) or "Miller" or "Wilke"  
   
          
    return dssp

In [9]:
def get_naccess(model2):
    
    "'naccess' is the name of the binary or executable"
    #run naccess for a given model and retrieve naccess properties    
    model = model2
    N_data = NACCESS(model, pdb_file=None, naccess_binary='naccess')
    return N_data

In [10]:
def get_hse(model2):
    
    model = model2    
    exp_ca = HSExposure.HSExposureCA(model)   #(CA_up, CA_down, PCB_angle)  
    exp_cb = HSExposure.HSExposureCB(model)   #(CA_up, CA_down, CA_CB_angle)  
    exp_cn = HSExposure.ExposureCN(model)     #no of CA atoms, in radius = 12
    
    hse_data = (exp_ca, exp_cb, exp_cn)
    
    return hse_data

In [11]:
def get_msms(model2):
    "'msms' is the name of the binary or executable"
    #run msms for a given model and retrieve msms properties   
    #run the python kernel with administrator privileges, or 
    # there might be a problem with tmp file creation
    model = model2
    rd = ResidueDepth(model, msms_exec = "msms")
    return rd

In [12]:
def hydro(res_name):
    """Returns the Kyte and Doolittle (1982) Hydrophobicity for a residue"""
    
    
    kyte={
    'I':4.5,
    'V':4.2,
    'L':3.8,
    'F':2.8,
    'C':2.5,
    'M':1.9,
    'A':1.8,
    'G':-0.4,
    'T':-0.7,
    'W':-0.9,
    'S':-0.8,
    'Y':-1.3,
    'P':-1.6,
    'H':-3.2,
    'E':-3.5,
    'Q':-3.5,
    'D':-3.5,
    'N':-3.5,
    'K':-3.9,
    'R':-4.5
    }
    
    
    if res_name in kyte:
        H = kyte[res_name]
             
    else:
        H = 'None'
    return H

In [13]:
def get_SS(S8):
    """8 to 3 state SS conversion"""
    if re.match('^[HGI]', S8): 
        S3 = "H"
    
    elif re.match('^[EB]', S8): 
        S3 = "E";


    elif re.match('^[TS]', S8) or re.match('[^A-Z]', S8):
        S3 = "C";	

    else:
        S3 = 0   
    
    return S3

In [14]:
def get_chain_Bfactor(chain):
    #Calculates the total, average and standard deviation of B-factors in a chain
    #'Enhancing subtilisin thermostability through a modified normalized B-factor 
    #analysis and loop-grafting strategy', Jrnl of Biological Chemistry (Tang et.al., 2019)
    B_total = 0
    N_res = 0 #No. of residues
    B_list = [] #stores B-factors for each residue in chain
    
    
    for res in chain:
        B_res = 0
        N_atom = 0
        N_res += 1
        for atom in res:
            B_res += atom.get_bfactor()
            N_atom += 1
            B_total += atom.get_bfactor()
        B_i = B_res/N_atom
        B_list.append(B_i)
     
        
        
    B_i_all=np.array(B_list, dtype=np.float64)
    B_i_sum = np.sum(B_i_all) #Not equal to B_total. A large difference is seen. But why?
    B_sd=np.std(B_i_all, dtype=np.float64)
    B_mean = np.mean(B_i_all, dtype=np.float64)
    B_stats = np.array([[N_res], [B_mean], [B_sd]])     
    
    
    return B_stats

In [15]:
def get_norm_Bfactor(chain, res, bstats):
    #Get the residue normalized B-factor
    #'Enhancing subtilisin thermostability through a modified normalized B-factor 
    #analysis and loop-grafting strategy', Jrnl of Biological Chemistry (Tang et.al., 2019)
    N_atom = 0
    B_res = 0
    
    N = bstats[0] #total no of residues
    mean = bstats[1]
    sd = bstats[2]
    
    
    for atom in chain[res]:
        B_res += atom.get_bfactor()
        N_atom += 1
    
    B_i = B_res/N_atom    
    B_norm = (B_i - mean)/sd
    B_norm =  B_norm[0].astype(float)
    
    #Computes normalized B-factor for a single residue
    return B_norm

In [16]:
# Read in Epitope Sites and Save to list. 
# Examples
# 1RU7 (seq1) - H1N1 refstruct
# 1HGD (seq1) - H3N2 refstruct
# 7DZW (seq1) - SARS-CoV2 refstruct

df = pd.read_csv(sitesin_fn,  sep=s2, header=None)

Sites = df.to_numpy() 
Sites = Sites.tolist() #read in columns 1-3

#prepare Sites collect epitope residue ID in for seq2 (query)
for i, row in enumerate(Sites):
    Sites[i].append([]) #append columns 4-5
    Sites[i].append([]) 


In [17]:
df2 = pd.read_csv(id_file, sep=s1, header = None)
PDB_list = df2.values.tolist()

In [ ]:
model1 = get_model(struct_id1, chain_id1)
chain1 = model1[chain_id1]
seq1 = get_seq(chain1, struct_id1)

#Names of 25 features + N_res (mapped residue's name)
F_names = ['Hydropathy','SS','RSA','PHI','PSI', 'N_res', 'all_atoms_abs','all_atoms_rel','side_chain_abs',
            'side_chain_rel','main_chain_abs','main_chain_rel','non_polar_abs','non_polar_rel',
            'all_polar_abs', 'all_polar_rel', 'CA_Up', 'CA_down', 'CA_PCB_Angle', 'CA_Up.1', 
            'CA_down.1', 'CA_CB_Angle', 'CA_Count_r12','Residue_Depth','CA_Depth','B_Norm']
E_Keys = ['PDBID','Chain','Year_Strain','Ref_Res_No','Epitope','Mapped_Res_No','Mapped_Res']

header_names = E_Keys + F_names

op2 = open(op_fn, 'w') 
print(*header_names, sep = c, file = op2)
count = 0

for ID in PDB_list:
    struct_id2 = ID[0].strip()
    chain_id2 =  ID[1].strip()
    print("Now processing %s Chain %s ..." %(struct_id2, chain_id2))
    model2 = get_model(struct_id2, chain_id2)
    chain2 = model2[chain_id2]
    seq2 = get_seq(chain2,  struct_id2)
    fafile = get_algn(seq1, struct_id1, chain_id1, seq2, struct_id2, chain_id2)
    Sites = get_map_res(fafile, chain1, chain2)
    dssp = get_dssp(model2, struct_id2, chain_id2)
    naccess = get_naccess(model2)
    [hse_ca, hse_cb, hse_cn] = get_hse(model2)
    bstats = get_chain_Bfactor(chain2)    
    msms = get_msms(model2)
    
    for j, row in enumerate(Sites):
        
        res_key = (chain_id2,(row[3]))       
        
        if res_key[1][1] != 'None': #check that an aligned residue exists, first.
            if res_key in dssp:
                dssp_row = [dssp[res_key][1], hydro(dssp[res_key][1]), get_SS(dssp[res_key][2]), dssp[res_key][3], dssp[res_key][4],dssp[res_key][5]]
            else: 
                dssp_row = ['None' for k in range(5)]
            if res_key in naccess:
                n_row = list(naccess[res_key].values())
            else:
                n_row = ['None' for k in range(11)]
                
            if res_key in hse_ca:
                ha_row = list(hse_ca[res_key])
            else: 
                ha_row = ['None' for k in range(3)]
                
            if res_key in hse_cb:
                
                hb_row = list(hse_cb[res_key])
            else:
                hb_row = ['None' for k in range(3)]
                
            if res_key in hse_cn:      
                hn_row = [hse_cn[res_key]]
                
            else:
                hn_row = 'None'
            if res_key in msms:     
                depth = list(msms[res_key])
            else:
                depth = ['None', 'None']
            if chain2.__contains__(res_key[1]):
                B_norm = get_norm_Bfactor(chain2, res_key[1], bstats)
                
            else:
                B_norm = 'None'                  
           
           
            data_row = dssp_row + n_row + ha_row + hb_row + hn_row + depth 
            data_row.append(B_norm)
            #print(ID, res_key, B_norm)
           
                    
        else:
                            
            data_row = ['None' for k in range(25)]
        
        print(*ID[:3], Sites[j][0], Sites[j][1], Sites[j][2], *data_row, sep = c, file = op2 )
       
    count+=1
        
        
   
   

#op.close()
print("Completed %d structures" %(count))
op2.close()

In [22]:
#uncoment to save as dataframe
sites_final_df = pd.read_csv(op_fn,  sep = c)
#sites_final_df.shape